# Sensor Fusion Tutorial

This notebook teaches you how to combine data from multiple sensors to create a more robust perception system.

## What You'll Learn

1. Understanding sensor fusion concepts
2. Combining LiDAR and camera data
3. Temporal fusion (combining data over time)
4. Building a simple fusion algorithm
5. Visualizing fused perception data

## Prerequisites

- Understanding of sensors (see [Building Custom Sensors](../building/building_custom_sensors.ipynb))
- Understanding of perception data structures
- Basic numpy knowledge

In [ ]:
import sys
sys.path.insert(0, '../../src')

import numpy as np
import matplotlib.pyplot as plt
from simple_autonomous_car import (
    Track,
    Car,
    CarState,
    GroundTruthMap,
    PerceivedMap,
    LiDARSensor,
    PerceptionPoints,
)

## Step 1: Understanding Sensor Fusion

Sensor fusion combines data from multiple sensors to:
- Reduce uncertainty
- Fill gaps in coverage
- Improve reliability
- Handle sensor failures

In [ ]:
# Create setup
track = Track.create_simple_track(length=80.0, width=40.0, track_width=5.0)
ground_truth_map = GroundTruthMap(track)
perceived_map = PerceivedMap(ground_truth_map)

start_point, start_heading = track.get_point_at_distance(0.0)
car = Car(initial_state=CarState(x=start_point[0], y=start_point[1], heading=start_heading, velocity=8.0))

# Add multiple sensors
front_lidar = LiDARSensor(
    ground_truth_map, perceived_map,
    max_range=30.0, angular_resolution=0.1,
    name="front_lidar",
    pose_ego=np.array([1.0, 0.0, 0.0])  # 1m forward
)

rear_lidar = LiDARSensor(
    ground_truth_map, perceived_map,
    max_range=20.0, angular_resolution=0.15,
    name="rear_lidar",
    pose_ego=np.array([-1.0, 0.0, np.pi])  # 1m back, facing rear
)

car.add_sensor(front_lidar)
car.add_sensor(rear_lidar)

print(f"✓ Setup complete with {len(car.sensors)} sensors")

## Step 2: Simple Fusion - Combine All Points

In [ ]:
def fuse_perception_simple(perception_data: dict, car_state: CarState) -> PerceptionPoints:
    """
    Simple fusion: combine all perception points from all sensors.
    """
    all_points = []
    
    for sensor_name, points in perception_data.items():
        if points is not None and len(points.points) > 0:
            # Convert to global frame if needed
            if points.frame != "global":
                points_global = points.to_global_frame(car_state).points
            else:
                points_global = points.points
            
            all_points.append(points_global)
    
    if len(all_points) == 0:
        return PerceptionPoints(points=np.array([]).reshape(0, 2), frame="global")
    
    fused_points = np.vstack(all_points)
    return PerceptionPoints(points=fused_points, frame="global")

# Test fusion
perception_data = car.sense_all(environment_data={"ground_truth_map": ground_truth_map})
fused = fuse_perception_simple(perception_data, car.state)

print(f"✓ Fused perception: {len(fused.points)} points")
print(f"  Front LiDAR: {len(perception_data['front_lidar'].points) if perception_data.get('front_lidar') else 0} points")
print(f"  Rear LiDAR: {len(perception_data['rear_lidar'].points) if perception_data.get('rear_lidar') else 0} points")

## Step 3: Temporal Fusion - Combine Over Time

In [ ]:
class TemporalFusion:
    """
    Temporal fusion: combine perception data over multiple time steps.
    """
    
    def __init__(self, max_history: int = 5, decay_factor: float = 0.8):
        self.max_history = max_history
        self.decay_factor = decay_factor
        self.history = []  # List of (points, timestamp) tuples
    
    def fuse(self, current_points: PerceptionPoints, timestamp: float) -> PerceptionPoints:
        """
        Fuse current perception with historical data.
        """
        # Add current points to history
        if current_points is not None and len(current_points.points) > 0:
            self.history.append((current_points.points.copy(), timestamp))
        
        # Keep only recent history
        if len(self.history) > self.max_history:
            self.history.pop(0)
        
        # Combine all historical points (with decay)
        all_points = []
        for i, (points, ts) in enumerate(self.history):
            age = len(self.history) - 1 - i
            weight = self.decay_factor ** age
            
            # For simplicity, just include all points (could weight by distance)
            if weight > 0.3:  # Only include recent enough points
                all_points.append(points)
        
        if len(all_points) == 0:
            return PerceptionPoints(points=np.array([]).reshape(0, 2), frame="global")
        
        fused_points = np.vstack(all_points)
        return PerceptionPoints(points=fused_points, frame="global")

# Test temporal fusion
temporal_fusion = TemporalFusion(max_history=3, decay_factor=0.7)

for step in range(5):
    perception_data = car.sense_all(environment_data={"ground_truth_map": ground_truth_map})
    fused_simple = fuse_perception_simple(perception_data, car.state)
    fused_temporal = temporal_fusion.fuse(fused_simple, step * 0.1)
    
    # Move car slightly
    car.update(0.1, acceleration=0.5, steering_rate=0.0)
    
    if step == 4:
        print(f"✓ Temporal fusion after {step+1} steps: {len(fused_temporal.points)} points")
        print(f"  Current step only: {len(fused_simple.points)} points")

## Step 4: Visualizing Fused Data

In [ ]:
from simple_autonomous_car.visualization import plot_track, plot_perception, plot_car

# Get current perception
perception_data = car.sense_all(environment_data={"ground_truth_map": ground_truth_map})
fused = fuse_perception_simple(perception_data, car.state)

# Visualize
fig, axes = plt.subplots(1, 2, figsize=(16, 8))

# Left: Individual sensors
ax = axes[0]
plot_track(track, ax=ax)
if perception_data.get('front_lidar'):
    plot_perception(perception_data['front_lidar'], car.state, ax=ax, frame="global", color="red", label="Front LiDAR")
if perception_data.get('rear_lidar'):
    plot_perception(perception_data['rear_lidar'], car.state, ax=ax, frame="global", color="orange", label="Rear LiDAR")
plot_car(car, ax=ax, show_heading=True)
ax.set_title("Individual Sensors")
ax.legend()

# Right: Fused data
ax = axes[1]
plot_track(track, ax=ax)
plot_perception(fused, car.state, ax=ax, frame="global", color="purple", label="Fused Perception")
plot_car(car, ax=ax, show_heading=True)
ax.set_title("Fused Perception")
ax.legend()

plt.tight_layout()
plt.show()

print("✓ Fusion visualization complete!")

## Summary

You've learned:

1. ✅ **Sensor fusion basics**: Combining data from multiple sensors
2. ✅ **Simple fusion**: Merge all perception points
3. ✅ **Temporal fusion**: Combine data over time
4. ✅ **Visualization**: Compare individual vs fused data

### Key Concepts

- **Spatial fusion**: Combine sensors at same time
- **Temporal fusion**: Combine data over time
- **Frame conversion**: Ensure all data in same frame
- **Weighting**: Can weight sensors by confidence

### Next Steps

- Build probabilistic fusion (Kalman filters)
- Build confidence-based weighting
- Build outlier rejection
- Build sensor failure detection